## Problem 4: What is the closest shopping center from your home / work? (5 points)

In the last problem you should find out the closest shopping center from two different locations a) your home and b) work place. 

**Steps**:

 - Create a txt-file called `activity_locations.txt` (use same formatting as in Problem 1) with two columns:
    - `id`: unique id for each row
    - `addr`:  address of your work and home (or any two addresses in the Helsinki Region.)
    
Save the text file into this repository.
    
 - Read those addresses using pandas and geocode the addresses.
 - Find out the nearest shopping center to these points using Shapely `nearest_points`.
 - Print out the name of the shopping center that is nearest to a) home and b) work. For example: `Shopping center closest to home: REDI`.

In [39]:
# read in input files
import pandas as pd

locs = pd.read_table('data/activity_locations.txt', sep=';')
locs

,id,addr
0,0,"Koirasaarentie 35, 00590 Helsinki"
1,1,"Gustaf Hällströmin katu 2, 00560 Helsinki"


In [2]:
# Geocode activity locations
from geopandas.tools import geocode

# Geocode the addresses using Nominatim
geo = geocode(locs['addr'], provider='nominatim', user_agent='autogis_2021', timeout=5)
geo

,geometry,address
0,POINT (25.01811 60.16924),"Koirasaarentie, Hopealaakso, Kruunuvuorenranta..."
1,POINT (24.96214 60.20501),"Kumpulan kampuskirjasto, 2, Gustaf Hällströmin..."


In [4]:
# Check coordinate reference system information (re-project/define if needed)
print(geo.crs)
geo.to_crs(epsg=3879, inplace=True)
print(geo.crs)

EPSG:4326
epsg:3879


In [5]:
# Find the closest shopping center for each activity location (closest shopping center from
# and closest shopping center from work) and print out the results:
import geopandas as gpd

centers = gpd.read_file('data/shopping_centers.shp')
centers

,address,id,name,addr,geometry
0,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",0,Itis,"Itäkatu 1-7, 00930 Helsinki, Finland",POINT (25504598.602 6677662.109)
1,"Salaattiasema, 14-20, Mannerheimintie, Keskust...",1,Forum,"Mannerheimintie 14-20, 00100 Helsinki, Finland",POINT (25496573.542 6672878.360)
2,"Iso Omena, 11, Piispansilta, Matinkylä, Suur-M...",2,Iso-Omena,"Piispansilta 11, 02230 Espoo, Finland",POINT (25485470.196 6672071.413)
3,"Lasten kappeli Arkki, 3-9, Leppävaarankatu, Sä...",3,Sello,"Leppävaarankatu 3-9, 02600 Espoo, Finland",POINT (25489491.076 6678322.265)
4,"Stockmann, 3, Vantaanportinkatu, Vantaanportti...",4,Jumbo,"Vantaanportinkatu 3, 01510 Vantaa",POINT (25497943.932 6686656.982)
5,"Silta, 5, Hermannin rantatie, Verkkosaari, Kal...",5,REDI,"Hermannin rantatie 5, 00580 Helsinki",POINT (25498837.156 6674909.983)
6,"Pasilansilta, Keski-Pasila, Pasila, Keskinen s...",6,Tripla,"Pasilansilta 11, 00520 Helsinki",POINT (25496166.681 6676094.626)


In [7]:
from shapely.ops import nearest_points

shopping_centers = centers["geometry"].unary_union

In [40]:
locs = geo.join(locs)[['id', 'addr', 'geometry']]

locs['nearest_shopping_center'] = locs['geometry'].map(lambda x: nearest_points(x, shopping_centers)[1])

In [41]:
locs

,id,addr,geometry,nearest_shopping_center
0,0,"Koirasaarentie 35, 00590 Helsinki",POINT (25501005.394 6672928.627),POINT (25498837.156 6674909.983)
1,1,"Gustaf Hällströmin katu 2, 00560 Helsinki",POINT (25497900.311 6676914.199),POINT (25496166.681 6676094.626)


In [42]:
locs = locs.merge(centers, how='left', \
                  left_on='nearest_shopping_center', \
                  right_on='geometry')[['id_x', 'addr_x', \
                                        'nearest_shopping_center', \
                                        'name']]
locs.rename(columns={'id_x': 'id', 'addr_x': 'addr', \
                     'name': 'nearest_shopping_center_name'}, inplace=True)

In [44]:
locs

,id,addr,nearest_shopping_center,nearest_shopping_center_name
0,0,"Koirasaarentie 35, 00590 Helsinki",POINT (25498837.156 6674909.983),REDI
1,1,"Gustaf Hällströmin katu 2, 00560 Helsinki",POINT (25496166.681 6676094.626),Tripla


Can you think of other application cases for the nearest neighbour analysis?